<div class="alert alert-success">
<h2> Комментарий ревьюера 4</h2>

Здорово, теперь ты все сделал правильно. Почти все) Смотри, доли должников - это число должников делить на общее число клиентов в группе, т.е. на число должников и число недолжников. Главное, что ты освоил данный метод. Все помарки исправлены, и теперь работа выполнена хорошо. Ты здорово поработал, молодец. Успехов в дальнейших проектах :)
    
Теперь можно отмечать не только новый год, но и засчитанный первый проект!
</div>

<div class="alert alert-success">
<h2> Комментарий ревьюера 3</h2>

Привет. Оформление комментариев по работе сохраняется. Только обозначим, что это третья итерация. 

</div>

<div class="alert alert-success">
<h2> Комментарий ревьюера 2</h2>

Привет еще раз. Спасибо, что доделал работу. Оформление комментариев по работе сохраняется. Только обозначим, что это вторая итерация. 

</div>

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Привет! Спасибо, что прислал задание:) Поздравляю с первым сданным проектом. Меня зовут Слепцов Артем и я буду проверять твой проект) Ты проделал большую работу над проектом, однако его еще можно улучшить. Будет здорово, если ты, надеюсь, не против, если я буду на ты, будешь отвечать на комментарии и участвовать в диалоге. 

Мои комментарии обозначены пометкой **Комментарий ревьюера**. Далее в файле ты сможешь найти их в похожих ячейках (если фон комментария <font color=green>зелёный</font> - всё сделано правильно. Рекомендации укажу таким же цветом; <font color=orange>оранжевый</font> - некритичные замечания. Если таких замечаний меньше трех - проект может быть принят без их отработки; <font color=red>красный</font> - нужно переделать). Не удаляй эти комментарии и постарайся учесть их в ходе выполнения данного проекта. 

</div>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div class="alert alert-warning">
<h2> Комментарий ревьюера</h2>

Правильно, что есть краткое вступление в работу, описание того, что надо делать. В работе следует приводить краткий план того, что надо сделать(если этот план имеется), а также информацию о входных данных: какие столбцы есть в таблице, их названия и какую информацию они несут. Также название работы лучше отображать в отдельной ячейке и делать крупный шрифт (заголовок). Так работа выглядит презентабельно. Подробнее смотри [методичку](https://www.notion.so/35d9c2f32ffa42f3b17010737aedca4b) по оформлению проектов.

</div>

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

<div class="alert alert-warning">
<h2> Комментарий ревьюера </h2>

Я заметил, что ячейки в твоей тетрадке начинаются не с 1. Перед отправкой работы рекомендую перезапускать ноутбук, чтобы убедиться, что все ячейки выполняются корректно.

</div>

Привет) Хорошо, буду иметь ввиду

<div class="alert alert-success">
<h2> Комментарий ревьюера 2</h2>

Перезапуск позволяет обнаружить и исправить скрытые в коде ошибки. 

</div>

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
print('-----------------------------------------------------------------------------')
display(data.columns)
print('-----------------------------------------------------------------------------')
display(data.head(10))
print('-----------------------------------------------------------------------------')
display(data[data['days_employed'].isnull()].head(10))
print('-----------------------------------------------------------------------------')
display(data.groupby('income_type').sum())
print('-----------------------------------------------------------------------------')
data['days_employed'] = abs(data['days_employed']) # Проверим работает ли все тоже только с положительными значениями (убедимся в правильности суждения о том, что: у тех кто работает отрицательный трудовой стаж)
display(data.groupby('income_type').sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
-----------------------------------------------------------------------------


Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

-----------------------------------------------------------------------------


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


-----------------------------------------------------------------------------


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


-----------------------------------------------------------------------------


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
income_type,,,,,,,
безработный,1,7.328273e+05,76,1,1,1,2.626795e+05
в декрете,2,-3.296760e+03,39,1,0,1,5.382913e+04
госслужащий,908,-4.460665e+06,59289,980,1251,86,2.242186e+08
компаньон,3044,-9.664447e+06,201862,3638,5250,376,9.264647e+08
пенсионер,509,1.256707e+09,227747,3524,3801,216,4.721299e+08
предприниматель,0,-5.208481e+02,85,0,1,0,4.991631e+05
сотрудник,7136,-2.329756e+07,442770,9447,10626,1061,1.616062e+09
студент,0,-5.787516e+02,22,0,4,0,9.820163e+04


-----------------------------------------------------------------------------


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
income_type,,,,,,,
безработный,1,7.328273e+05,76,1,1,1,2.626795e+05
в декрете,2,3.296760e+03,39,1,0,1,5.382913e+04
госслужащий,908,4.460665e+06,59289,980,1251,86,2.242186e+08
компаньон,3044,9.664447e+06,201862,3638,5250,376,9.264647e+08
пенсионер,509,1.256707e+09,227747,3524,3801,216,4.721299e+08
предприниматель,0,5.208481e+02,85,0,1,0,4.991631e+05
сотрудник,7136,2.329756e+07,442770,9447,10626,1061,1.616062e+09
студент,0,5.787516e+02,22,0,4,0,9.820163e+04


<div class="alert alert-warning">
<h2> Комментарий ревьюера </h2>

Таблицы на печать выводи с помощью метода display. print переводит данные в строку, а нам этого не надо. 

</div>

Тут все исправил

<div class="alert alert-success">
<h2> Комментарий ревьюера 2</h2>

Согласись, такой вывод таблиц куда приятнее для глаз)

</div>

### Вывод

1. Смущает столбец days_employed. Отрицательный трудовой стаж и + тип данных, если это количество дней, то должно быть целое число. Получается так что у тех кто не работает (безработный и пенсионер) положительный трудовой стаж у всех остальных отрицательный...
2. Столбец education. Разный регистр в строках - нужно привести к нижнему;
3. С названиями столбцов все хорошо;
4. C остальными типами данных тоже все гуд;
5. Есть связь между столбцами days_employed и total_income. Возможно неофициальное трудоустройство или неофициальный заработок или отсутствие работы на сегодняшний день.

<div class="alert alert-success">
<h2> Комментарий ревьюера </h2>

Подумай также о возможных причинах появления пропусков, а также о том, являются ли они случайными или нет. Можно высказать предварительные варианты их обработки. 

Первый взгляд на таблицу выполнен.

</div>

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
print(data.isnull().sum())# В таблице отсутствуют данные как и предпологалось ранее в двух столбцах

pd.options.mode.chained_assignment = None # Отключение предупреждения

income_type_dropdup = data['income_type'].drop_duplicates(inplace = False).reset_index(drop = True)

for i in range(len(income_type_dropdup)):
    data['days_employed'].loc[(data['income_type'] == income_type_dropdup[i]) & (data['days_employed'].isnull())] = data.groupby('income_type').mean()['days_employed'][i]
    data['total_income'].loc[(data['income_type'] == income_type_dropdup[i]) & (data['total_income'].isnull())] = data.groupby('income_type').mean()['total_income'][i]
    
print(data.isnull().sum())



children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


<div class="alert alert-success">
<h2> Комментарий ревьюера </h2>

С предупреждениями также стоит разобраться. Иногда их бывает слишком много, поэтому важно уметь их скрывать. В этом тебе поможет библиотека warnings. Попробуй найти подходящий метод и убрать предупреждения.  

</div>

Убрал предупреждение установив значение chained_assignment = None, спасибо за подсказку). Если я правильно понял данное предупреждение связано с тем, что в моем случае не всегда получится заменить данные, но не понял в каком случае это будет невозможно, буду благодарен, если приведешь мне пару примеров, чтобы я знал где можно так делать, а где нет.

<div class="alert alert-success">
<h2> Комментарий ревьюера 2</h2>

Приведу ссылку на данное предупреждение - [ссылка](https://www.dataquest.io/blog/settingwithcopywarning/). Там довольно подробно раскрыта тема.

</div>

### Вывод

Отсутствующие данные в столбцах days_employed и total_income относятся к количественным, поэтому для каждой категории столбца income_type было найдено среднее значение и все пропуски были заменены. После проверки данных пустых значений не было выявлено

<div class="alert alert-success">
<h2> Комментарий ревьюера </h2>

По логике ты обработал пропуски правильно. Однако для расчетов среднего лучше сразу использовать метод mean, а не sum/count.

</div>

Да, действительно код смотрится гораздо лаконичнее.

<div class="alert alert-success">
<h2> Комментарий ревьюера 2</h2>

👍 

</div>

### Замена типа данных

In [4]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
print(data.head(10))

   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   Среднее             1   
3         3           4124         32   среднее             1   
4         0         340266         53   среднее             1   
5         0            926         27    высшее             0   
6         0           2879         43    высшее             0   
7         0            152         50   СРЕДНЕЕ             1   
8         2           6929         35    ВЫСШЕЕ             0   
9         0           2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

### Вывод

Нет смысла считать рассматривать количество рабочих дней, в виде дробного числа, поэтому тип данных был изменен на целочисленный. Cтолбец total_income так-же имеет смысл привести к целому типу данных т.к нет смысла работать с "копейками", они не будут влиять на результат, но повлияют на скорость выполнения операций. Метод astype применяется в случае, когда необходимо менять тип данных у целых чисел, строк или дробных чисел, для времени и даты существуют другие методы. Т.к необходимо было сделать изменение данных из float -> int, был выбран именно этот метод.

<div class="alert alert-success">
<h2> Комментарий ревьюера </h2>

Исходя из общей информации о таблице мы видим, что два столбца имеют вещественный тип данных, который и надо заменить на целочисленный. Эта замена произведена верно. Также хорошо бы освоить метод to_numeric() для будущих работ. 

</div>

pd.to_numeric(data['days_employed'], downcast = 'integer') Пытался сделать таким образом, почему-то не получилось... Остается тип float64

<div class="alert alert-success">
<h2> Комментарий ревьюера 2</h2>

to_numeric приводит строку '0' в число 0. В данном случае мы его использовать не можем, но он пригодится для будущих проектов.  

</div>

### Обработка дубликатов

In [5]:
data['education'] = data['education'].str.lower() # Для поиска дубликатов первое, что необходимо сделать, это перевести к одному регистру (верхнему/нижнему) столбец education

print(data[data.duplicated()].count())
data = data.drop_duplicates().reset_index(drop = True)
print(data[data.duplicated()].count())

children            71
days_employed       71
dob_years           71
education           71
education_id        71
family_status       71
family_status_id    71
gender              71
income_type         71
debt                71
total_income        71
purpose             71
dtype: int64
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Вывод

Метод drop_duplicates позволил нам избавится от дубликатов. Данный метод был выбран т.к он удаляет все строки дубликаты, а это именно то, что нам нужно. Так же была выполнена проверка data[data.duplicated()].count() на количство дубликатов в таблице до и после выполнения метода drop_duplicates. Дубликаты могут появляться абсолютно при любых обстоятельствах, возможно кто-то случайно продублировал данные. Вывод: успешно удалены все дубликаты.

<div class="alert alert-success">
<h2> Комментарий ревьюера </h2>

Дубликаты вызывают смещение финальных результатов, поэтому их и надо удалять. Конечно, в данных нет уникального идентификатора, однако надо уметь удалять дубликаты. Метод drop_duplicates() использован верно.

</div>

### Лемматизация

In [6]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
uniqe_arr_purpose = ' '.join(data['purpose'].unique()) # Берем все уникальные значения в столбце purpose и лемматизируем их
lemmas = m.lemmatize(uniqe_arr_purpose) # Получаем список лемматизированых слов

array_unique_lemmas = []

for element in Counter(lemmas).keys(): # Выводим все уникальные значения лемматизированного списка
    if len(element) > 3:
        array_unique_lemmas.append(element)     

print(array_unique_lemmas)
my_array_unique_lemmas = ['жил', 'автомо', 'образовани', 'свадьб', 'недвижим', 'строительство']

['покупка', 'жилье', 'приобретение', 'автомобиль', 'дополнительный', 'образование', 'сыграть', 'свадьба', 'операция', 'проведение', 'семья', 'недвижимость', 'коммерческий', 'жилой', 'строительство', 'собственный', 'подержать', 'свой', 'заниматься', 'сделка', 'получение', 'высокий', 'профильный', 'сдача', 'ремонт']


### Вывод

Были выделены уникальные элементы столбца с целями кредита и по каждой цели проведена лемматизация. Теперь при необходимости можно выяснить на что чаще всего клиенты берут кредит или использовать полученные данные для любых других целей. 

<div class="alert alert-warning">
<h2> Комментарий ревьюера </h2>

Круто, ты сделал самую сложную часть проекта. Использовал нужные методы, и даже провел группировку с помощью собственной функции. Только стоит провести выбор главных целей кредитов, на которые мы затем и классифицируем клиентов. 

</div>

Я создал список с главными целями путем перебора ключей словаря и с помощью условия убрал не нужные слова, длинна которых меньше трех 

<div class="alert alert-success">
<h2> Комментарий ревьюера 2</h2>

Стоило вывести уникальные цели кредитов из таблицы. Их не так много. Так бы ты подтвердил выбор конкретно таких главных целей. 
    

</div>

### Категоризация данных

In [7]:
data_income_group = pd.qcut(data['total_income'], 4)
income_group_unique = pd.Series(pd.qcut(data['total_income'], 4).unique()).sort_values(ascending=False)
income_group_unique = income_group_unique.astype('str')

array_group = []

for i in range(income_group_unique.shape[0]):
    array_group.append(income_group_unique[i].split(',')[0][1:])
    array_group[i] = int(float(array_group[i]))

array_group.sort(reverse=True)   

def definitionIncome(income):
    if income > array_group[0]:
        return 'Высокий доход'
    elif array_group[1] <= income <= array_group[0]:
        return 'Доход выше среднего'
    elif array_group[2] <= income <= array_group[1]:
        return 'Доход ниже среднего'
    else:
        return 'Низкий доход'

data['person_income_value'] = data['total_income'].apply(definitionIncome)

rest_log_family = data[['family_status_id', 'children']] 
rest_dict_family = data[['family_status_id', 'family_status']]
rest_dict_family = rest_dict_family.drop_duplicates().reset_index(drop=True)

rest_log_education = data[['education_id', 'total_income']] 
rest_dict_education = data[['education_id', 'education']]
rest_dict_education = rest_dict_education.drop_duplicates().reset_index(drop=True)
print('----------------------------------')
print(rest_log_family.groupby('family_status_id').sum().sort_values('children',ascending=False))
print(rest_dict_family)
print('----------------------------------')
print(rest_log_education.groupby('education_id').mean().sort_values('total_income',ascending=False))
print(rest_dict_education)
print('----------------------------------')







----------------------------------
                  children
family_status_id          
0                     7890
1                     2126
4                      806
3                      547
2                      215
   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  Не женат / не замужем
----------------------------------
               total_income
education_id               
0             200867.662667
2             177864.549731
4             174749.833333
1             151015.491695
3             131067.152482
   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень
----------------------------------


### Вывод

Все клиенты банка успешно разбиты на две категории: те у кого заработок меньше среднего и те у кого заработок выше среднего

<div class="alert alert-danger">
<h2> Комментарий ревьюера </h2>

Исходя из чего выбран именно такой способ категоризации столбца с доходами? Так мы получаем всего 2 группы. Результат анализа будет недостаточен. Лучше использовать разбиение по квантилям данных (метод qcut). Тогда группы получатся одинакового размера. Или же можно узнать в интернете процентный состав общества по доходам и затем уже категоризовать столбец с доходами по найденному соотношению с помощью персентилей. 

</div>

С помощью метода qcut получил 4 категории заработка, код получился достаточно громоздким из-за большого количества преобразований, если есть способ упростить код, было бы круто чтобы ты показал, как)

<div class="alert alert-success">
<h2> Комментарий ревьюера 2</h2>

Здорово, что ты освоил данный метод. Так группы у нас получаются равного размера. Мы можем по ним построить выводы. 

</div>

<div class="alert alert-danger">
<h2> Комментарий ревьюера </h2>

В данных есть словари. Попробуй их обнаружить и выделить. Это довольно распространенный способ хранения информации. Мне важно понять, что он тобой освоен. Заметь, что в основной таблице мы имеем по 2 параметра с образованием и семейным статусом. Именно по ним и стоит выделить словари. 

</div>

На самом деле комбинации словарей могут быть абсолютно любыми. Например статус семьи -> средний заработок, статус семьи -> количество отработанных дней (sum, count, mean) и т.д

<div class="alert alert-success">
<h2> Комментарий ревьюера 2</h2>

Конечно, ты прав. Главное, что ты выделил словари. Теперь мы можем убрать текстовые параметры из основной таблицы, что увеличит эффективность работы с данными. 

</div>

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
for i in range(1,6):
    print('Процент не выплативших кредит по отношению к выплатившим с ({} - им/мя/ю) ребенком составляет {:.1%}:'.format(i, data['children'].loc[(data['children'] == i) & (data['debt'] == 1)].count() / data['children'].loc[(data['children'] == i) & (data['debt'] == 0)].count()))

print('-----------------------')
print('        КТО ОНИ?!      ')
print('Процент не выплативших кредит по отношению к выплатившим с 20 детьми {:.1%}:'.format(data['children'].loc[(data['children'] == 20) & (data['debt'] == 1)].count() / data['children'].loc[(data['children'] == 20) & (data['debt'] == 0)].count()))
print(data.loc[data['children'] == 20].count()['children']) # Либо это мега артефакт, либо нужно остановить этого человека... 20 ДЕТЕЙ!? Там ещё есть -1 ребенок, это я уже похже обнаружил

Процент не выплативших кредит по отношению к выплатившим с (1 - им/мя/ю) ребенком составляет 10.2%:
Процент не выплативших кредит по отношению к выплатившим с (2 - им/мя/ю) ребенком составляет 10.4%:
Процент не выплативших кредит по отношению к выплатившим с (3 - им/мя/ю) ребенком составляет 8.9%:
Процент не выплативших кредит по отношению к выплатившим с (4 - им/мя/ю) ребенком составляет 10.8%:
Процент не выплативших кредит по отношению к выплатившим с (5 - им/мя/ю) ребенком составляет 0.0%:
-----------------------
        КТО ОНИ?!      
Процент не выплативших кредит по отношению к выплатившим с 20 детьми 11.8%:
76


### Вывод

Таким образом мы видим что количество людей у которых нет детей и выплативших в срок кредит больше на 51% чем людей у которых есть дети. Делаем вывод, что налиие детей негативно влияет на выплату кредита в срок

<div class="alert alert-danger">
<h2> Комментарий ревьюера </h2>

Категоризовать данные по числу детей необязательно. Разбиение есть уже в исходных данных. Все-таки разница в возврате кредита у семей с одним ребенком или тремя детьми будет. Так ты только уменьшаешь объем имеющейся информации. 
    
Посмотри на результаты по полным данным. 

</div>

Т.е получается те у кого нет детей и у кого их 3 и 5 самые ответственные родители. Но если серьезно, то четкой закономерности не прослеживается. Думаю что влияет все же не количество детей влияет на оплату кредита а доход и другие составляющие... Как мы видим у вас может быть и 20 детей :D но они отличаются на один процент с теми у кого 1 ребенок. Все таки нет четкой закономерности нет, но я думаю это логично

<div class="alert alert-success">
<h2> Комментарий ревьюера 2</h2>

Да, такое вполне может быть. 

</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
data.groupby('family_status').agg(['sum', 'count','mean'])['debt']

,sum,count,mean
family_status,,,
Не женат / не замужем,274,2810,0.097509
в разводе,85,1195,0.071130
вдовец / вдова,63,959,0.065693
гражданский брак,388,4151,0.093471
женат / замужем,931,12339,0.075452


### Вывод

Как бы это не звучало, но, кажется, жениться / выходить за муж - это верный путь к не возврату, как к нормальной жизни так и кредита...

<div class="alert alert-success">
<h2> Комментарий ревьюера </h2>
  
Будет здорово, если ты освоишь метод [agg](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html), который можно применить после группировки и передать в него лист с аггрегирующими функциями: sum - для подсчета числа должников, count - для подсчета общего числа клиентов в группах и mean - для нахождения доли должников в каждой из групп. 

</div>

Спасибо за совет, забыл про этот метод, он действительно очень наглядно показывает результаты вычислений

<div class="alert alert-success">
<h2> Комментарий ревьюера 2</h2>

Здорово, что ты его освоил. 

</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
print('{:.1%}'.format(data['person_income_value'].loc[(data['person_income_value'] == 'выше среднего') & (data['debt'] == 1)].count() / data['person_income_value'].loc[(data['person_income_value'] == 'выше среднего') & (data['debt'] == 0)].count()))
print('{:.1%}'.format(data['person_income_value'].loc[(data['person_income_value'] == 'ниже среднего') & (data['debt'] == 1)].count() / data['person_income_value'].loc[(data['person_income_value'] == 'ниже среднего') & (data['debt'] == 0)].count()))
#Случайно запустил... но было так

nan%
nan%


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in long_scalars
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in long_scalars
  


### Вывод

Исходя из полученного результата можно сделать вывод, что чем больше доход, тем меньше людей, которые не закрывают кредит вовремя, что очень даже логично

<div class="alert alert-success">
<h2> Комментарий ревьюера </h2>
  
Посмотри на результаты при ином способе категоризации. 

</div>

In [11]:
print('{:.1%}'.format(data['person_income_value'].loc[(data['person_income_value'] == 'Высокий доход') & (data['debt'] == 1)].count() / data['person_income_value'].loc[(data['person_income_value'] == 'Высокий доход') & (data['debt'] == 0)].count()))
print('{:.1%}'.format(data['person_income_value'].loc[(data['person_income_value'] == 'Доход выше среднего') & (data['debt'] == 1)].count() / data['person_income_value'].loc[(data['person_income_value'] == 'Доход выше среднего') & (data['debt'] == 0)].count()))
print('{:.1%}'.format(data['person_income_value'].loc[(data['person_income_value'] == 'Доход ниже среднего') & (data['debt'] == 1)].count() / data['person_income_value'].loc[(data['person_income_value'] == 'Доход ниже среднего') & (data['debt'] == 0)].count()))
print('{:.1%}'.format(data['person_income_value'].loc[(data['person_income_value'] == 'Низкий доход') & (data['debt'] == 1)].count() / data['person_income_value'].loc[(data['person_income_value'] == 'Низкий доход') & (data['debt'] == 0)].count()))

7.5%
9.5%
9.6%
8.6%


Исходя из полученного результата можно сделать вывод, что чем больше доход, тем меньше людей, которые не закрывают кредит вовремя, что очень даже логично. Вывод получился тот же

- Как разные цели кредита влияют на его возврат в срок?

### Вывод

In [16]:
data['category_purpose'] = ''

for element_lem in my_array_unique_lemmas:
    for j in range(data.shape[0]):
        if element_lem in data['purpose'][j]:
            data['category_purpose'][j] = element_lem

data_pivot = data.pivot_table(index = 'category_purpose', columns = 'debt', values='gender', aggfunc = 'count') # Использовал твой метод

data_pivot['category_purpose_percent'] = data_pivot[1] / data_pivot[0] * 100
    
data_pivot

debt,0,1,category_purpose_percent
category_purpose,,,
автомо,3903,403,10.325391
жил,4152,308,7.418112
недвижим,4143,330,7.965243
образовани,3643,370,10.156464
свадьб,2138,186,8.699719
строительство,1734,144,8.304498


In [19]:
# Код ревьюера
data.pivot_table(index = 'category_purpose', columns = 'debt', values='gender', aggfunc = 'count')

debt,0,1
category_purpose,,
автомо,3903,403
жил,4152,308
недвижим,4143,330
образовани,3643,370
свадьб,2138,186
строительство,1734,144


<div class="alert alert-danger">
<h2> Комментарий ревьюера 2</h2>

Круто, что ты использовал метод сводных таблиц. Однако код выдает ошибку. Пожалуйста, посмотри, что пошло не так. 

</div>

Видимо вчера ночью забыл нажать ctrl + s. Надеюсь +- правильно сделал. Но тут только можно оценку глазами сделать. Есть ли способ как то каждую категорию оценить через pivot_table?

<div class="alert alert-danger">
<h2> Комментарий ревьюера 3</h2>

Не совсем все так. Смотри, ниже твоего кода я привел более короткий. Нам не нужно знать, как распределяется доля долгов внутри большой цели кредитов, т.е. нам не важно, что доля долгов "на авто" 5%, а "на покупку авто" - 10%, условно. Это все одна большая тема "авто", которую и стоит рассматривать. Далее рассчитывай долю должников в каждой из групп. Именно этот показатель нас и интересует. Я привел нужную таблицу. Тебе осталось в ней рассчитать доли должников в каждой из групп. 

</div>

Ааа... теперь я понял, т.е для того чтобы оценить каждую категорию отдельно необходимо просто один индекс указать. Теперь мы видим, что процент оплаты кредита не вовремя меньше всего у целей связанных с жилищными вопросами (покупка, преобретение, ремонт и т.д.). Делаем вывод, что категории с данными целями больше всего будут подходить для одобрения кредита. Спасибо тебе большое, ты мне очень много объяснил, ты крутой ревьювер)

Изучив полученные данные, можно сделать вывод, что в процентном соотношении люди, которые берут кредит для решения жилищных вопросов лучше всех справляются с своевременной оплатой кредита

<div class="alert alert-danger">
<h2> Комментарий ревьюера </h2>

Перед анализом данного параметра стоит провести его категоризацию. Согласись, "сделка с автомобилем" и "на покупку автомобиля" - это операции из одной глобальной темы "авто". Выдели главные цели кредитов на этапе лемматизации, затем уже проведи категоризацию клиентов по выделенным целям и проанализируй зависимость целей и возврата кредита. 

</div>

<div class="alert alert-danger">
<h2> Комментарий ревьюера </h2>

На данном шаге рекомендуется использовать метод сводных таблиц (pivot_table()). Однако он является аналогом метода groupby(). Изучи метод сводных таблиц и примени его хотя бы в одном пункте анализа.

</div>

Исходя из представленной таблицы можно сказать, что больше всего возврата кредита в срок у тех, кто берет его для организации свадьбы, что логично ведь, на свадьбе дарят деньги которыми можно затем закрыть кредит. В остальном значения сильно не отличаются.

### Шаг 4. Общий вывод

В ходе работы над данным проектом были выполнены следующие основные задачи:

1. Полностью изучена таблица
2. Таблица очищена от пропусков
3. Значения в столбцах приведены к нужному типу данных
4. Таблица очищена от дубликатов
5. Проведена лемматизация и выделены основные цели кредита
6. Проведена категоризация по уровню дохода, а также созданы два словаря

Выводы:

1. Зависимость наличия детей от выплаты кредита в срок. Да закономерность в глобальном смысле есть, мы выяснили, что те у кого есть дети оплачивают кредит вовремя чаще чем те, у кого детей нет, это может быть связано с тем, что дети - это ответственность и не оплата кредита, это все же не очень хорошо. Лишние проблемы взрослым людям с ребенком абсолютно не нужны;
2. Далее это зависимость семейного положения. Как оказалось, люди которые находятся в гражданском браке или не замужем/женаты так же выплачивают чаще кредит в срок, возможно это не клеится с первого взгляда с первым пунктом, но наличие детей в гражданском браке, это совершенно нормальная история, что касается не замужем/женаты это так же не может 100% указывать на отсутствие или наличие детей;
3. Конечно же следует обращать внимание на доход клиента, чем больше доход, тем больше вероятность выплаты кредита в срок;
4. Как оказалось процент оплаты кредита не вовремя меньше всего у категорий связанных с жилищными вопросами.

Итог: Клиент - наличие детей, гражданский брак или статус не женат/замужем, высокий доход, цель кредита - любая операция связанная с жилищными вопросами (покупка, преобретение и т.д)

<div class="alert alert-danger">
<h2> Комментарий ревьюера </h2>

Финальный вывод и есть главный результат твоей работы. Стоит писать его подробно по результатам проведенной работы. В нем можно приводить полученные в ходе работы значения. Также можно расписать все, что было сделано в работе.

</div>

<div class="alert alert-success">
<h2> Комментарий ревьюера 2</h2>

Выводы описаны грамотно. Приведены ответы на главные вопросы проекта. В выводах можно приводить полученные ранее значения, правильно. Здорово, что по каждому пункту работы приведен вывод.     

</div>

<div class="alert alert-warning">
<h2> Комментарий ревьюера </h2>

- В начале работы следует описывать данные, с которыми работаешь;
- Категоризацию по доходу стоит делать иначе. Смотри данный пункт подробнее;
- Выдели словари в данных;
- Категоризуй клиентов по целям кредитов;
- Изучи и используй в анализе метод сводных таблиц;
- Финальный вывод стоит писать подробно.

Ты проделал большую работу, молодец! Исправь отмеченные выше помарки. Также есть достаточное число желтых комментариев. Думаю, исправить их тебе не составит труда. Жду твою работу :)

</div>

<div class="alert alert-success">
<h2> Комментарий ревьюера </h2>

Ты поработал со всеми помарками, молодец. Однако в коде есть ошибка, которую следует поправить. Как сделаешь это, присылай работу снова :)
    
С наступающими праздниками!

</div>

Спасибо) И тебя! Надеюсь бог любит троицу и на этом проект ты примешь, ну а если нет, то я ещё прокачаюсь)

<div class="alert alert-success">
<h2> Комментарий ревьюера 3</h2>

Остался последний момент при работе со сводной таблицей. Надеюсь, ты справишься с указанной помаркой. Данный метод важно освоить хорошо. Он важен для дальнейших проектов :)
</div>

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.